In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path

from sdm_eurec4a.visulization import set_custom_rcParams, label_from_attrs
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

default_colors = set_custom_rcParams()

import matplotlib.pyplot as plt

In [ ]:
data_dir = Path(
    "/home/m/m301096/CLEO/data/output_v4.0/coalbure_condensation_large/combined/eulerian_dataset_combined.nc"
)
ds = xr.open_dataset(data_dir)
ds = ds.isel(time=slice(0, -1))

In [ ]:
microphysics = (
    # 'null_microphysics',
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    # 'coalbure_condensation_large',
)
compare_ds = xr.open_dataset(
    f"/home/m/m301096/CLEO/data/output_v4.0/condensation/combined/eulerian_dataset_combined.nc"
)
compare_ds = compare_ds.isel(time=slice(0, -1))
compare_da = compare_ds["massdelta_condensation"]


for mp, color in zip(microphysics, default_colors):

    data_dir = Path(f"/home/m/m301096/CLEO/data/output_v4.0/{mp}/combined/eulerian_dataset_combined.nc")
    ds = xr.open_dataset(data_dir)
    ds = ds.isel(time=slice(0, -1))

    da = ds["massdelta_condensation"]

    da_mean, da_std = mean_and_stderror_of_mean(
        da.sel(time=slice(2000, 3400)),
        dims="time",
    )
    compare_da_mean, compare_da_std = mean_and_stderror_of_mean(
        compare_da.sel(time=slice(2000, 3400)),
        dims="time",
    )

    # da_mean = da_mean - compare_da_mean
    # da_std = np.sqrt(da_std**2 + compare_da_std**2)

    m, s = mean_and_stderror_of_mean(data=da_mean, dims="cloud_id", data_std=da_std)

    plt.plot(
        da_mean.T,
        da_mean["gridbox"],
        color=color,
        alpha=0.4,
    )
    plt.plot(
        m,
        m["gridbox"],
        color=color,
        linestyle="-",
        label=mp,
    )
    plt.fill_betweenx(
        m["gridbox"],
        m - s,
        m + s,
        color=color,
        alpha=0.3,
    )

# plt.xlim(-0.2e-6, 1e-7)
# plt.xscale("symlog", linthresh=1e-9)
plt.legend()